In [2]:
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jacqueville,CI,5.2052,-4.4146,80.51,86,78,5.19,broken clouds
1,1,Hilo,US,19.7297,-155.0900,68.40,95,34,1.01,scattered clouds
2,2,Hermanus,ZA,-34.4187,19.2345,64.90,83,100,4.12,overcast clouds
3,3,Port Blair,IN,11.6667,92.7500,86.04,62,40,9.22,scattered clouds
4,4,Talaya,RU,61.3833,152.7500,-28.88,88,96,1.23,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 65
What is your desired maximum temperature for your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,68.40,95,34,1.01,scattered clouds
6,6,Mar Del Plata,AR,-38.0023,-57.5575,66.87,85,0,14.00,clear sky
13,13,Puerto Ayora,EC,-0.7393,-90.3518,72.66,87,56,5.77,broken clouds
16,16,Port Elizabeth,ZA,-33.9180,25.5701,74.12,61,0,7.27,clear sky
18,18,Vaini,TO,-21.2000,-175.2000,78.96,100,20,8.05,few clouds
19,19,Tuatapere,NZ,-46.1333,167.6833,66.81,76,56,2.39,broken clouds
25,25,Arraial Do Cabo,BR,-22.9661,-42.0278,69.51,94,100,3.60,overcast clouds
30,30,Atuona,PF,-9.8000,-139.0333,78.44,77,17,17.07,few clouds
32,32,Belmonte,BR,-15.8631,-38.8828,77.50,87,58,2.64,broken clouds
34,34,Pleasant Point,NZ,-44.2667,171.1333,68.29,83,1,4.07,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,68.40,95,34,1.01,scattered clouds
6,6,Mar Del Plata,AR,-38.0023,-57.5575,66.87,85,0,14.00,clear sky
13,13,Puerto Ayora,EC,-0.7393,-90.3518,72.66,87,56,5.77,broken clouds
16,16,Port Elizabeth,ZA,-33.9180,25.5701,74.12,61,0,7.27,clear sky
18,18,Vaini,TO,-21.2000,-175.2000,78.96,100,20,8.05,few clouds
...,...,...,...,...,...,...,...,...,...,...
656,656,Porto Velho,BR,-8.7619,-63.9039,75.25,94,0,2.30,clear sky
661,661,Cockburn Town,TC,21.4612,-71.1419,77.59,73,0,19.64,clear sky
665,665,Bukama,CD,-9.2000,25.8500,71.42,94,100,0.69,overcast clouds
666,666,Melipilla,CL,-33.7000,-71.2167,75.54,97,19,1.03,few clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,68.40,scattered clouds,19.7297,-155.0900,
6,Mar Del Plata,AR,66.87,clear sky,-38.0023,-57.5575,
13,Puerto Ayora,EC,72.66,broken clouds,-0.7393,-90.3518,
16,Port Elizabeth,ZA,74.12,clear sky,-33.9180,25.5701,
18,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
19,Tuatapere,NZ,66.81,broken clouds,-46.1333,167.6833,
25,Arraial Do Cabo,BR,69.51,overcast clouds,-22.9661,-42.0278,
30,Atuona,PF,78.44,few clouds,-9.8000,-139.0333,
32,Belmonte,BR,77.50,broken clouds,-15.8631,-38.8828,
34,Pleasant Point,NZ,68.29,clear sky,-44.2667,171.1333,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,68.40,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Mar Del Plata,AR,66.87,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
13,Puerto Ayora,EC,72.66,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Port Elizabeth,ZA,74.12,clear sky,-33.9180,25.5701,39 On Nile Guest House
18,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))